# 1. Import Dependencies

In [1]:
# Tutorial by Nicholas Renotte, https://www.youtube.com/watch?v=doDUihpj6ro
# Written manually by g33k
# Where mentioned, improvements done by g33k
# Used to develop ASL dataset by g33k

In [2]:
!pip install tensorflow==2.3.1 tensorflow-gpu==2.3.1 opencv-python mediapipe sklearn matplotlib

In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MediaPipe Holistics

In [4]:
mp_holistic = mp.solutions.holistic     # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Color conversion BGR to RGB
    image.flags.writeable = False                  # Set image to unwriteable
    results = model.process(image)                 # Make Predictions
    image.flags.writeable = True                   # Set image to writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Color conversion RGB to BGR
    return image, results

In [6]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [7]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=1),
                            mp_drawing.DrawingSpec(color=(0,255,255), thickness=1, circle_radius=1)
    )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10),thickness = 2, circle_radius = 4),
                             mp_drawing.DrawingSpec(color=(80,44,121),thickness = 2, circle_radius = 2)
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76),thickness = 2, circle_radius = 4),
                             mp_drawing.DrawingSpec(color=(121,44,250),thickness = 2, circle_radius = 2)
                             )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66),thickness = 2, circle_radius = 4),
                             mp_drawing.DrawingSpec(color=(245,66,230),thickness = 2, circle_radius = 2)
                             )

In [8]:
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()

        # Make detection
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Set to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

# 3. Extract Keypoint Values

In [9]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [10]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [11]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [12]:
extract_keypoints(results)[:10]

array([ 0.58606553,  0.44074762, -1.89627683,  0.99972516,  0.61099917,
        0.36420447, -1.84557223,  0.9994604 ,  0.62636888,  0.36016217])

# 4. Setup Folders for Collection

In [13]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data')

# Signs to be detected
actions = np.array(['do-do', 'cap-log', 'hello'])

# 30 videos worth of data per sign 
no_sequences = 30

# 30 frames per video 
sequence_length = 30

# Folder start
start_folder = 0

for action in actions: 
    dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(1,no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
        except:
            pass

In [14]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [15]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [16]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

2022-03-04 18:45:34.405985: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [17]:
label_map = {label:num for num, label in enumerate(actions)}

In [18]:
label_map

{'do-do': 0, 'cap-log': 1, 'hello': 2}

In [19]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [20]:
X = np.array(sequences)

In [21]:
y = to_categorical(labels).astype(int)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05)

# 7. Build and Train LSTM Neural Network

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [24]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir = log_dir)

2022-03-04 18:45:48.226859: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-03-04 18:45:48.280448: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1


In [34]:
model = Sequential()
model.add(LSTM(64, return_sequences = True, activation = 'relu', input_shape = (30, 1662)))
model.add(LSTM(128, return_sequences = True, activation = 'relu'))
model.add(LSTM(64, return_sequences = False, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(actions.shape[0], activation = 'softmax'))

In [35]:
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

In [27]:
model.fit(X_train, y_train, epochs = 2000, callbacks = [tb_callback])

Epoch 1/2000
1/3 [=========>....................] - ETA: 0s - loss: 1.1162 - categorical_accuracy: 0.2812WARNING:tensorflow:From /home/g33k/ASLactRec/ASLrecog/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


2022-03-04 18:46:03.894660: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.


3/3 [==============================] - 1s 321ms/step - loss: 1.1114 - categorical_accuracy: 0.3412


2022-03-04 18:46:04.539234: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: Logs/train/plugins/profile/2022_03_04_18_46_04
2022-03-04 18:46:04.627929: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to Logs/train/plugins/profile/2022_03_04_18_46_04/EDITH.trace.json.gz
2022-03-04 18:46:04.672324: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: Logs/train/plugins/profile/2022_03_04_18_46_04
2022-03-04 18:46:04.672482: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for memory_profile.json.gz to Logs/train/plugins/profile/2022_03_04_18_46_04/EDITH.memory_profile.json.gz
2022-03-04 18:46:04.673861: I tensorflow/python/profiler/internal/profiler_wrapper.cc:111] Creating directory: Logs/train/plugins/profile/2022_03_04_18_46_04Dumped tool data for xplane.pb to Logs/train/plugins/profile/2022_03_04_18_46_04/EDITH.xplane.pb
Dumped tool data for

Epoch 2/2000
3/3 [==============================] - 0s 98ms/step - loss: 3.4015 - categorical_accuracy: 0.3412
Epoch 3/2000
3/3 [==============================] - 0s 88ms/step - loss: 1.7773 - categorical_accuracy: 0.3412
Epoch 4/2000
3/3 [==============================] - 0s 97ms/step - loss: 1.1804 - categorical_accuracy: 0.3412
Epoch 5/2000
3/3 [==============================] - 0s 95ms/step - loss: 3.6098 - categorical_accuracy: 0.2706
Epoch 6/2000
3/3 [==============================] - 0s 85ms/step - loss: 1.2181 - categorical_accuracy: 0.4118
Epoch 7/2000
3/3 [==============================] - 0s 87ms/step - loss: 1.2969 - categorical_accuracy: 0.2353
Epoch 8/2000
3/3 [==============================] - 0s 87ms/step - loss: 1.1176 - categorical_accuracy: 0.3059
Epoch 9/2000
3/3 [==============================] - 0s 86ms/step - loss: 1.0975 - categorical_accuracy: 0.3412
Epoch 10/2000
3/3 [==============================] - 0s 99ms/step - loss: 1.0935 - categorical_accuracy: 0.4000


...

Epoch 1646/2000
3/3 [==============================] - 0s 86ms/step - loss: 0.1355 - categorical_accuracy: 0.9529
Epoch 1647/2000
3/3 [==============================] - 0s 86ms/step - loss: 0.1152 - categorical_accuracy: 0.9529
Epoch 1648/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.0938 - categorical_accuracy: 0.9647
Epoch 1649/2000
3/3 [==============================] - 0s 89ms/step - loss: 0.1038 - categorical_accuracy: 0.9647
Epoch 1650/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.1090 - categorical_accuracy: 0.9529
Epoch 1651/2000
3/3 [==============================] - 0s 90ms/step - loss: 0.1003 - categorical_accuracy: 0.9412
Epoch 1652/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.0959 - categorical_accuracy: 0.9647
Epoch 1653/2000
3/3 [==============================] - 0s 88ms/step - loss: 0.1025 - categorical_accuracy: 0.9529
Epoch 1654/2000
3/3 [==============================] - 0s 90ms/step - loss: 0.1251 - cat

3/3 [==============================] - 0s 87ms/step - loss: 0.1263 - categorical_accuracy: 0.9529
Epoch 1718/2000
3/3 [==============================] - 0s 86ms/step - loss: 0.1227 - categorical_accuracy: 0.9529
Epoch 1719/2000
3/3 [==============================] - 0s 86ms/step - loss: 0.1436 - categorical_accuracy: 0.9647
Epoch 1720/2000
3/3 [==============================] - 0s 86ms/step - loss: 0.1256 - categorical_accuracy: 0.9294
Epoch 1721/2000
3/3 [==============================] - 0s 86ms/step - loss: 0.1076 - categorical_accuracy: 0.9647
Epoch 1722/2000
3/3 [==============================] - 0s 88ms/step - loss: 0.1282 - categorical_accuracy: 0.9647
Epoch 1723/2000
3/3 [==============================] - 0s 86ms/step - loss: 0.2181 - categorical_accuracy: 0.9412
Epoch 1724/2000
3/3 [==============================] - 0s 88ms/step - loss: 0.3725 - categorical_accuracy: 0.8706
Epoch 1725/2000
3/3 [==============================] - 0s 85ms/step - loss: 0.6074 - categorical_accurac

3/3 [==============================] - 0s 87ms/step - loss: 0.0998 - categorical_accuracy: 0.9529
Epoch 1789/2000
3/3 [==============================] - 0s 86ms/step - loss: 0.0536 - categorical_accuracy: 0.9882
Epoch 1790/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.0989 - categorical_accuracy: 0.9412
Epoch 1791/2000
3/3 [==============================] - 0s 86ms/step - loss: 0.0848 - categorical_accuracy: 0.9647
Epoch 1792/2000
3/3 [==============================] - 0s 88ms/step - loss: 0.0584 - categorical_accuracy: 0.9882
Epoch 1793/2000
3/3 [==============================] - 0s 86ms/step - loss: 0.0527 - categorical_accuracy: 0.9882
Epoch 1794/2000
3/3 [==============================] - 0s 88ms/step - loss: 0.0505 - categorical_accuracy: 0.9882
Epoch 1795/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.0447 - categorical_accuracy: 1.0000
Epoch 1796/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.0494 - categorical_accurac

3/3 [==============================] - 0s 113ms/step - loss: 0.1588 - categorical_accuracy: 0.9412
Epoch 1860/2000
3/3 [==============================] - 0s 98ms/step - loss: 0.1574 - categorical_accuracy: 0.9412
Epoch 1861/2000
3/3 [==============================] - 0s 107ms/step - loss: 0.1660 - categorical_accuracy: 0.9412
Epoch 1862/2000
3/3 [==============================] - 0s 102ms/step - loss: 0.1948 - categorical_accuracy: 0.9294
Epoch 1863/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.2012 - categorical_accuracy: 0.9294
Epoch 1864/2000
3/3 [==============================] - 0s 113ms/step - loss: 0.1657 - categorical_accuracy: 0.9294
Epoch 1865/2000
3/3 [==============================] - 0s 103ms/step - loss: 0.1537 - categorical_accuracy: 0.9294
Epoch 1866/2000
3/3 [==============================] - 0s 88ms/step - loss: 0.1468 - categorical_accuracy: 0.9294
Epoch 1867/2000
3/3 [==============================] - 0s 93ms/step - loss: 0.1392 - categorical_ac

3/3 [==============================] - 0s 90ms/step - loss: 0.2226 - categorical_accuracy: 0.9412
Epoch 1931/2000
3/3 [==============================] - 0s 93ms/step - loss: 0.2327 - categorical_accuracy: 0.9294
Epoch 1932/2000
3/3 [==============================] - 0s 88ms/step - loss: 0.2305 - categorical_accuracy: 0.9294
Epoch 1933/2000
3/3 [==============================] - 0s 81ms/step - loss: 0.2221 - categorical_accuracy: 0.9294
Epoch 1934/2000
3/3 [==============================] - 0s 82ms/step - loss: 0.2120 - categorical_accuracy: 0.9294
Epoch 1935/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.2038 - categorical_accuracy: 0.9412
Epoch 1936/2000
3/3 [==============================] - 0s 88ms/step - loss: 0.1948 - categorical_accuracy: 0.9412
Epoch 1937/2000
3/3 [==============================] - 0s 105ms/step - loss: 0.1805 - categorical_accuracy: 0.9412
Epoch 1938/2000
3/3 [==============================] - 0s 111ms/step - loss: 0.1600 - categorical_accur

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
__________________________________________________

# 8. Make Predictions

In [29]:
res = model.predict(X_test)

In [30]:
actions[np.argmax(res[0])]

'do-do'

In [31]:
actions[np.argmax(y_test[0])]

'do-do'

# 9. Save Weights

In [32]:
model.save('commands.h5')

In [33]:
del model

In [36]:
model.load_weights('commands.h5')

# 10. Evaluation Using Confusion Matrix and Accuracy

In [37]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [38]:
yhat = model.predict(X_test)

In [39]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [40]:
ytrue

[0, 1, 2, 2, 0]

In [41]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 0],
        [0, 2]],

       [[4, 0],
        [0, 1]],

       [[3, 0],
        [0, 2]]])

In [42]:
accuracy_score(ytrue, yhat)

1.0

# 11. Test in Real Time

In [43]:
from scipy import stats

In [44]:
colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.putRectangle(output_frame, (0, 60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
    return output_frame

In [57]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.7

cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
    
        # Read feed
        ret, frame = cap.read()

        # Make detection
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # 2. Prediction Logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))

        # 3. Viz Logic
        if np.unique(predictions[-10:])[0]==np.argmax(res): 
            if res[np.argmax(res)] > threshold: 
                    
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])


            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Set to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>


IndexError: index 0 is out of bounds for axis 0 with size 0

In [58]:
cap.release()
cv2.destroyAllWindows()